# <span style="color:brown"> **Hands-on Deep Learning – Knowledge Distiallation for Image Classification**

_This hands-on was created by Thomas Grenier, CREATIS_

thomas.grenier@creatis.insa-lyon.fr

Application and code adapted from :
-    https://keras.io/examples/vision/knowledge_distillation/
-   **https://keras.io/examples/keras_recipes/better_knowledge_distillation/**
-    https://github.com/sayakpaul/FunMatch-Distillation


# <span style="color:brown"> **Introduction Knowledge Distillation**

Knowledge distillation (Hinton et al.  https://arxiv.org/abs/1503.02531 ) refers to double training: one teacher and one, or more, students.
The teacher is classically a larger model than the students' models. The teacher is trained first, then it will assist the student during its training.

The objective can be to compress the teacher to reduce its memory footprint and inference time, still preserving the high performances of the large model.
Other applications exist such as unsupervised training of students.

An interesting discussion on the good receipt for knowledge distillation can be found in the Bayer works "Knowledge distillation: A good teacher is patient and consistent" (https://arxiv.org/abs/2106.05237).

It is the fourth part of the classification hands-on: we will use the dataset 'D' and use on Knowledge Distillation to compress model. 
Here, we don't use data augmentation strategies (MixUp) nor learning rate schedule (warmup cosine learning rate).

> Let's go!

# <span style="color:brown"> **A - Setup**

### A1 - First, import a few common modules, ensure MatplotLib plots figures inline, and functions for converting data and displaying reports:

In [ ]:
# Common imports
# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

import numpy as np

!pip install tqdm
from tqdm import tqdm

!pip install scikit-learn
from sklearn import datasets, metrics

### A2- We import TensorFlow 2

In [ ]:
import tensorflow as tf
print(tf.__version__)

> _**If no error occurs, ie the version of TensorFlow is printed, your working environment is ok and you can go to the next part,
> else ... call an assistant for help!**_

# <span style="color:brown"> **B - Read the data**

<span style="color:red">
    

We now use a function to do most of the work on data.

In [ ]:
from tools import ReadHandsOnData
from tools import DisplayRGBImages

Some important values and initializations:

In [ ]:
desired_height = 64
desired_width  = 64
input_shape = (desired_height,desired_width,1)
input_shape_rgb = (desired_height,desired_width,3)

class_names = ['PD-A', 'PD-C', 'PD-S', 'T1-A', 'T1-C', 'T1-S', 'T2-A', 'T2-C', 'T2-S']

# Definition of constants
num_classes = len(class_names)

### <span style="color:brown"> **B1 - Load data**

This dataset is based on the same volumes but 64x64 random crops are done around the center of the original image.

In [ ]:
dataset="dlss_classification_D"

X_train_rgb, y_train_sparse, X_val_rgb, y_val_sparse = ReadHandsOnData('train_val', randomize_image=True, validation_ratio=0.2, desired_height=desired_height, desired_width=desired_width, class_names=class_names, file_link = dataset)
X_test_rgb, y_test_sparse = ReadHandsOnData('test', randomize_image=True, validation_ratio=0, desired_height=desired_height, desired_width=desired_width, class_names=class_names, file_link = dataset)
DisplayRGBImages(X_val_rgb, y_val_sparse, nb_x = 5, nb_y = 5, class_names=class_names)

In [ ]:
nb_train = X_train_rgb.shape[0]
X_train_rgb = X_train_rgb[0:nb_train,]
y_train_sparse = y_train_sparse[0:nb_train,]

nb_val = X_val_rgb.shape[0]
X_val_rgb = X_val_rgb[0:nb_val,]
y_val_sparse = y_val_sparse[0:nb_val,]

nb_test = X_test_rgb.shape[0]
X_test_rgb = X_test_rgb[0:nb_test,]
y_test_sparse = y_test_sparse[0:nb_test,]

### <span style="color:brown"> **B2 - Convert the data shape and type**


The _png_ files are RGB, we use them as is as the teacher will be based on ResNet (which needs RGB inputs).

As usual, we divide intensity values by 255 to bind the range to [0;1] and convert the labels to categorical.


In [ ]:
# converting the three data set : rgb to gray scale images, then normalization
X_train = X_train_rgb[:,:,:,0].astype(np.float32)/255
X_train = np.expand_dims(X_train,-1)

X_val = X_val_rgb[:,:,:,0].astype(np.float32)/255
X_val = np.expand_dims(X_val,-1)

X_test = X_test_rgb[:,:,:,0].astype(np.float32)/255
X_test = np.expand_dims(X_test,-1)

print(" Shape : ", X_train.shape, X_val.shape, X_test.shape)
print(" Type  : %s  %s %s"%(X_train.dtype, X_val.dtype, X_test.dtype))
print(" Max   : %d %d %d"%(X_train.max(), X_val.max(), X_test.max()) )
print(" Min   : %d %d %d"%(X_train.min(),X_val.min(), X_test.min()) )

y_train = tf.keras.utils.to_categorical(y_train_sparse)
y_val   = tf.keras.utils.to_categorical(y_val_sparse)
y_test  = tf.keras.utils.to_categorical(y_test_sparse)

print(" Shape : ", y_train.shape, y_val.shape, y_test.shape)

In [ ]:
nb_x = 5
nb_y = 5
plt.figure(figsize=(15,15*nb_x/nb_y), dpi=100)
for i in range(1, nb_x * nb_y +1):
    plt.subplot(nb_x, nb_y ,i)
    plt.imshow(X_train[i-1], aspect="equal", cmap="gray", interpolation="nearest")
    plt.axis("off")  
    plt.title(class_names[np.argmax(y_train[i-1])])

> _**All done, go to the classification step!**_

# <span style="color:brown"> **D - CNN Classification with distillation**


### Function for displaying reports

In [ ]:
# Usage example : ClassificationReport(model, X_test, y_test_cat , class_names)
def ClassificationReport(model, X_test, y_test_cat, labels_name):  
    # predict 
    print("Prediction on test images")
    predicted = model.predict(X_test)
    y_pred = np.argmax(predicted, axis=-1)
    y_test = np.argmax(y_test_cat, axis=-1)
    
    print("Classification report :\n%s\n"
         % (metrics.classification_report(y_test, y_pred)) )
    cm = metrics.confusion_matrix( y_test, y_pred )
    disp = metrics.ConfusionMatrixDisplay(confusion_matrix=cm,  display_labels=labels_name)
    disp.plot()
    disp.figure_.suptitle("Confusion Matrix")
    plt.show()

In [ ]:
class Distiller(tf.keras.Model):
    def __init__(self, student, teacher):
        super().__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        """ Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super().compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)

            # Compute scaled distillation loss from https://arxiv.org/abs/1503.02531
            # The magnitudes of the gradients produced by the soft targets scale
            # as 1/T^2, multiply them by T^2 when using both hard and soft targets.
            distillation_loss = (
                self.distillation_loss_fn(
                    tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                    tf.nn.softmax(student_predictions / self.temperature, axis=1),
                )
                * self.temperature**2
            )

            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        for metric in self.metrics:
            metric.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        for metric in self.metrics:
            metric.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results

In [ ]:
# Definition of variables that you will have to play with during the session to study their influence
# Variables related to the FCN architecture
nb_mlp_neurons = 16    # Number of neurons for one layer
nb_cnn_neurons = 16    # number of feature maps

# Variables related to the optimization process
batch_size  = 32       # Number of images for each batch during the training process 
nb_epoch_teacher = 20  # number of epochs used during the teacher training process 
nb_epoch_student = 20  # number of epochs used during the student training process
learning_rate = 0.0001  # learning rate used during the training process

### <span style="color:brown"> **C2 - Teacher model**

We finetune a resnet that will be the "teacher".

In [ ]:
# Create the teacher
if 'teacher' in locals(): 
    print("deleting teacher model")
    del teacher

inputs = tf.keras.layers.Input(shape=input_shape_rgb)
#x = tf.keras.ops.cast(inputs, dtype="float32")  # TF >= 2.16
x = tf.cast(inputs, dtype="float32") # TF < 2.16
outputs_preproc = tf.keras.applications.resnet_v2.preprocess_input(x)  # pixels values are changed to [-1;1]

# Define the ResNet architecture and dowload weights trained on imagenet
baseModel = tf.keras.applications.ResNet50V2(
    weights="imagenet", # use "imagenet" to use imagenet resnet weights, or None (!! no " ") for random initialization of network parameters
    include_top=False,  # don't include the fully-connected layers (classification part) at the top of ResNet
    input_tensor=tf.keras.layers.Input(shape=input_shape_rgb))

outputs_base = baseModel(outputs_preproc)
baseModel_preproc = tf.keras.Model(inputs=[inputs], outputs=[outputs_base]) 

headModel = tf.keras.layers.Flatten(name = "flatten")(outputs_base)
headModel = tf.keras.layers.Dense(nb_mlp_neurons, activation = "relu")(headModel)
outputs_head = tf.keras.layers.Dense(num_classes)(headModel)

teacher = tf.keras.Model(inputs=[inputs], outputs=outputs_head, name="teacher")

# allow baseModel weights to be trainable ? (if True, the training step will be long i.e. 1 to 3 minutes per epoch)
baseModel.trainable = False
teacher.summary()


In [ ]:
# Train teacher as usual
teacher.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.CategoricalAccuracy()],
)

# Train and evaluate teacher on data.
baseModel.trainable = False
teacher.fit(X_train_rgb, y_train,  validation_data=(X_val_rgb, y_val), batch_size=batch_size, epochs=nb_epoch_teacher//2)
baseModel.trainable = True
teacher.fit(X_train_rgb, y_train,  validation_data=(X_val_rgb, y_val), batch_size=batch_size, epochs=nb_epoch_teacher//2)

In [ ]:
# Compute the model's overall accuracy on the test dataset 
test_loss, test_accuracy = teacher.evaluate(X_test_rgb, y_test, verbose=1)
print('TEACHER Test accuracy:', test_accuracy)

# Compute and display standard classification metrics, i.e. precision, recall, f1-score, support, and confusion matrix 
ClassificationReport(teacher, X_test_rgb, y_test, class_names)

### <span style="color:brown"> **C3 - Student model**

We create a small student model that should produce results similar to the ones of the teacher.

In [ ]:
# Create the student
if 'student' in locals(): 
    print("deleting student model")
    del student
    
student = tf.keras.Sequential(
  [
    tf.keras.layers.Input(shape=input_shape_rgb),
    tf.keras.layers.Conv2D(nb_cnn_neurons//2, 3, activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(nb_cnn_neurons, 3, activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(nb_cnn_neurons*2, 3, activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(nb_mlp_neurons, activation='relu'), 
    tf.keras.layers.Dense(num_classes, activation = None)
  ],
  name="student")


# Clone student for later comparison
student_scratch = tf.keras.models.clone_model(student)
student.summary()

In [ ]:
# Initialize and compile distiller
distiller = Distiller(student=student, teacher=teacher)
distiller.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
    metrics=[tf.keras.metrics.CategoricalAccuracy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=3,
)

# Distill teacher to student
distiller.fit(X_train_rgb, y_train, validation_data=(X_val_rgb, y_val), batch_size=batch_size,  epochs=nb_epoch_student)

In [ ]:
# Evaluate student on test dataset
#test_loss, test_accuracy = distiller.evaluate(X_test_rgb, y_test, verbose=1)
distiller.evaluate(X_test_rgb, y_test, verbose=1)
print('DISTILLATION STUDENT Test accuracy:')

# Compute and display standart classication metrics, i.e. precision, recall, f1-score, support and confusion matrix 
ClassificationReport(distiller.student, X_test_rgb, y_test, class_names)

### <span style="color:brown"> **C4 - Student model, without teacher**

What happens if we train the small student model alone ?

In [ ]:
# Train student as done usually
student_scratch.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.CategoricalAccuracy()],
)

# Train and evaluate the student model from scratch.
student_scratch.fit(X_train_rgb, y_train,  validation_data=(X_val_rgb, y_val), batch_size=batch_size,  epochs=nb_epoch_student)
test_loss, test_accuracy = student_scratch.evaluate(X_test_rgb, y_test)
print('STUDENT ALONE Test accuracy:', test_accuracy)

# Compute and display standard classification metrics, i.e. precision, recall, f1-score, support and confusion matrix 
ClassificationReport(student_scratch, X_test_rgb, y_test, class_names)

### <span style="color:red"> **Questions**
-    Which network has the best performance? Is it normal?
-    Compare the number of parameters for teacher and student models
-    What happens if the parameter alpha of the distiller is set to 0 ? Is the 'y' (ie the true labels) needed? Can you imagine applications using that tuning? What will be the hypotheses to garantee the student models learn well?

### <span style="color:brown"> **Now : 1- shutdown the kernel**
(menu --> Kernel --> "Shut down kernel")

### <span style="color:brown"> **Now : 2- go to next notebook!**